### important: whenever you make changes to an imported notebook you need to select "restart the kernel" in this notebook to load the changes then rerun the cells

In [ ]:
# ----------

TIMER_ON = True
SHOW_PREPROCESSING_RESULTS = True
SHOW_INTERMEDIATE_RESULTS = True
SHOW_DEBUG_RESULTS = False

# ----------

REFERENCE_MISSION = 2

# ALL
# mission_numbers = [REFERENCE_MISSION, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14]
# start_index = 0
# new_missions = []

# DEMO
# need to include mission 2 for histogram normalization and alignment
# for differencing of only latter missions, can remove mission 2
mission_numbers = [REFERENCE_MISSION, 10, 11, 12, 13, 14]
start_index = 0
new_missions = [15, 16]

# ---------- Anomaly Detection

MASK_SETTINGS_ALEX = (75, 175, 1, 2, 7, 1) # (lower limit, upper limit, erode radius, erode iterations, dilate radius, dilate iterations)

GRASS_SENSITIVITY = 45

HEATMAP_ANOMALY_SCORES = [0, 1]

# ---------- Heatmap

NUM_BUCKETS = 10
HEATMAP_SENSITIVITY_MIN = 15
HEATMAP_SENSITIVITY_MAX = 37

# ---------- Visualization

BBOX_COLORS = [(0, 0, 255), (255, 0, 0)] # red, blue
NUM_BBOX_ITERATIONS = 2
VOTING_SCHEME = 'MAJORITY' # AND, OR, MAJORITY
MAJORITY_SUCCESS = 2 # used for majority, number of detection methods that need to agree

# ----------

In [ ]:
import import_ipynb

import AlexFunctions as AF
import JoshFunctions as JF
import KyleFunctions as KF

import LoadData as LD
import ImagePreprocessing as IP
import VisualizeResults as VR

import numpy as np
import time
import sys

# Preprocess New Mission

In [ ]:
if len(new_missions) > 0:
    for new_mission in new_missions:
        if new_mission in mission_numbers:
            sys.exit('Mission number is already included.')

        if IP.run_image_preprocessor(mission_number=new_mission, save_results=True, display_results=SHOW_PREPROCESSING_RESULTS):
            mission_numbers.append(new_mission)
        else:
            sys.exit('Data for mission number ' + str(new_mission) + ' does not exist.')

print('Mission Numbers:', mission_numbers)

# Load Data

In [ ]:
if len(mission_numbers) < 2:
    sys.exit('Minimum of 2 missions required.')

In [ ]:
start = time.time()

# load all of the images and compute normalized versions (bgr, hsv, histogram)
data = LD.run_load_data(mission_numbers)

# original color images
mission_images_bgr = data[0][start_index:]
grass_images_bgr = data[1][start_index:]
building_images_bgr = data[2][start_index:]

# histogram normalized color images
histogram_mission_images_bgr = data[3][start_index:]
histogram_grass_images_bgr = data[4][start_index:]
histogram_building_images_bgr = data[5][start_index:]

# original hsv images
mission_images_hsv = data[6][start_index:]
grass_images_hsv = data[7][start_index:]
building_images_hsv = data[8][start_index:]

# histogram normalized hsv images
histogram_mission_images_hsv = data[9][start_index:]
histogram_grass_images_hsv = data[10][start_index:]
histogram_building_images_hsv = data[11][start_index:]

mission_numbers = mission_numbers[start_index:]
print('Mission Numbers:', mission_numbers)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

In [ ]:
if SHOW_INTERMEDIATE_RESULTS:
    AF.plot_images(image_list=mission_images_bgr)
    AF.plot_images(image_list=grass_images_bgr)
    AF.plot_images(image_list=building_images_bgr)
    
    AF.plot_images(image_list=histogram_mission_images_bgr)
    AF.plot_images(image_list=histogram_grass_images_bgr)
    AF.plot_images(image_list=histogram_building_images_bgr)
    
    AF.plot_images(image_list=mission_images_hsv)
    AF.plot_images(image_list=grass_images_hsv)
    AF.plot_images(image_list=building_images_hsv)
    
    AF.plot_images(image_list=histogram_mission_images_hsv)
    AF.plot_images(image_list=histogram_grass_images_hsv)
    AF.plot_images(image_list=histogram_building_images_hsv)

# Anomaly Detection + Grass Region Heatmap

In [ ]:
start = time.time()

# Alex - anomaly detection
anomaly_masks_1 = AF.anomaly_detection_1(mission_numbers, mission_images_hsv, histogram_mission_images_hsv, mask_settings=MASK_SETTINGS_ALEX, display_images=SHOW_DEBUG_RESULTS)

# Kyle - anomaly detection
anomaly_masks_2 = KF.anomaly_detection(grass_images_bgr, GRASS_SENSITIVITY)

# Kyle - heatmap + anomaly detection
heatmap, heatmap_percents, _, color_buckets, _, raw_anomaly_masks_3, anomaly_masks_3 = KF.score_missions(histogram_grass_images_bgr, NUM_BUCKETS, HEATMAP_ANOMALY_SCORES, HEATMAP_SENSITIVITY_MIN, HEATMAP_SENSITIVITY_MAX)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

In [ ]:
if SHOW_INTERMEDIATE_RESULTS:
    AF.plot_images(image_list=anomaly_masks_1)
    AF.plot_images(image_list=anomaly_masks_2)
    AF.plot_images(image_list=anomaly_masks_3)
    AF.plot_images(image_list=heatmap)

# Roof Anomaly

In [ ]:
start = time.time()

roof_masks, roof_anomaly_percentages = JF.findRoofAnomalies(histogram_building_images_hsv)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

In [ ]:
if SHOW_INTERMEDIATE_RESULTS:
    AF.plot_images(roof_masks)

# Visualize Results

In [ ]:
start = time.time()

VR.run_visualize_results(mission_numbers, [anomaly_masks_1, anomaly_masks_2, anomaly_masks_3], histogram_mission_images_bgr, NUM_BBOX_ITERATIONS, BBOX_COLORS, heatmap, heatmap_percents, roof_masks, roof_anomaly_percentages, color_buckets, voting_scheme=VOTING_SCHEME, majority_success=MAJORITY_SUCCESS, display_images=False)

if TIMER_ON: print('Duration: %.3f sec' % (time.time()-start))

In [ ]:
print("List the bucket values separated by commas that you would like to see plotted i.e 1,2,5")
temp_list = input()
temp_list = temp_list.split(",")
bucket_list = []
for each in temp_list:
    bucket_list.append(int(each)) #each is a string so convert to integer

VR.display_heatmap_change_scores(mission_numbers, np.array(heatmap_percents), color_buckets, scores=bucket_list)